In [1]:
# from diffusers import StableDiffusionXLPipeline
from pipelines.SDXL_pipeline import StableDiffusionXLPipeline
# from pipelines.SDUP_pipeline import StableDiffusionUpscalePipeline
# from pipelines.ddim_pipeline import StableDiffusionDDIMInvPipeline
from diffusers.image_processor import VaeImageProcessor

from diffusers.utils import pt_to_pil, numpy_to_pil
import torch
from IPython.display import display
import numpy as np
from PIL import Image
import PIL
image_processor=VaeImageProcessor()


[2023-09-18 10:47:45,774] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-0.9", torch_dtype=torch.float32, variant="fp32", use_safetensors=True
    # "stabilityai/stable-diffusion-xl-base-0.9", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
# pipe=pipe.to("cuda")
pipe.enable_model_cpu_offload()

/home-local/kwang-local/miniconda3/envs/floydkai/lib/python3.11/site-packages/diffusers/pipelines/pipeline_utils.py:1353: FutureWarning: You are trying to load the model files of the `variant=fp32`, but no such modeling files are available.The default model files: {'text_encoder_2/pytorch_model.bin', 'vae/diffusion_pytorch_model.bin', 'text_encoder/model.safetensors', 'vae/diffusion_pytorch_model.safetensors', 'unet/diffusion_pytorch_model.safetensors', 'unet/diffusion_pytorch_model.bin', 'text_encoder/pytorch_model.bin', 'text_encoder_2/model.safetensors'} will be loaded instead. Make sure to not load from `variant=fp32`if such variant modeling files are not available. Doing so will lead to an error in v0.22.0 as defaulting to non-variantmodeling files is deprecated.
  deprecate("no variant default", "0.22.0", deprecation_message, standard_warn=False)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
img_pth='images/cat_hq.jpg'

_inv_raw_image = Image.open(img_pth).convert("RGB").resize((1024,1024))
inv_raw_image = image_processor.preprocess(_inv_raw_image)

# image_ = image_processor.postprocess(inv_raw_image, output_type='pt')
# inv_raw_image,inv_raw_image_ = pil_to_numpy_torch(_inv_raw_image)

In [ ]:
# ### NOTE: 1,3,1024,1024. Scale -1,1. Pytorch Tensor
# (inv_raw_image+1)/2
# inv_raw_image

## 1st stage compress and decompress

In [4]:
generator = torch.manual_seed(0)
with torch.no_grad():
    latent = pipe.prepare_image_latents(inv_raw_image.cuda(), 1, pipe.vae.dtype, 'cuda',generator=generator)

In [5]:
with torch.no_grad():
    # image_pt = pipe.decode_latents_pt(latent)
    image = pipe.decode_latents(latent)

/disk1/users/kwang-local/ctang/deepfloyd/pipelines/SDXL_pipeline.py:544: FutureWarning: The decode_latents method is deprecated and will be removed in a future version. Please use VaeImageProcessor instead
  warnings.warn(


In [ ]:
# ((image_processor.denormalize(image_pt) - image_processor.denormalize(inv_raw_image))**2).mean()

### Save to PATH

In [9]:
PATH = 'SUXD.png'
numpy_to_pil(image)[0].save(PATH)